In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import LLM
llm = LLM(
    model = "gemini/gemini-2.0-flash",
    temperature=0.1
)
print(llm.call("do you know about GVPCE(A), visakhapatnam. Just say yes or no"))

Yes



In [4]:
original_email = ''' looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNBOT allows. ping me if any blockers.
'''

from crewai.tools import BaseTool 

class ReplaceJargonsTool(BaseTool):
    name: str = "Replace Jargons"
    description: str = "Replace Jargons with more specific terms"

    def _run(self, email:str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamp project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standup assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }

        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")

        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations detected."

jt = ReplaceJargonsTool()
print(jt.run(original_email))


Using Tool: Replace Jargons
Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'
Consider replacing 'TAS' with 'technical architecture stack'
Consider replacing 'SDS' with 'Smart Data Syncer'
Consider replacing 'ping' with 'reach out'


In [5]:
from crewai import Agent, Task, Crew    

email_assistant = Agent(
    role = "Email Assistant Agent",
    goal = "Improve emails and make them sound professional and clear",
    backstory = "A highly experienced communication expert skilled in professional email writing",
    tools = [jt],
    verbose= True,
    llm = llm
)



✅ What Happens When You Assign Tools to an Agent
When you pass tools=[jt] while defining an Agent, you are saying:

"This agent has access to this tool. Use it if and when needed."

🧠 But How Does the Agent Know When to Use the Tool?
It depends entirely on:

Your prompt or task (does it suggest the need for the tool?)

The tool's description (description= field) — which helps the LLM decide which tool is relevant

The agent’s own reasoning via LLM (e.g., GPT or Gemini)

🔗 Behind the Scenes:
Let’s say your task is:

“Make this email more professional and replace any abbreviations or unclear internal terms.”

The agent (powered by an LLM) sees:

Your email

The tool name: "Replace Jargons"

The tool description: "Replace Jargons with more specific terms"

Now, the agent thinks:

Hmm, this message has terms like TAS, PRX, SYNBOT, ping.
I have a tool that replaces jargons. I should use it.

So it chooses to call jt.run() under the hood.

If none of the email content matches any tool's description, then the agent won’t call that tool.

🤖 What If You Have 4 Tools, but Only 2 Should Be Used?
That’s exactly how CrewAI is designed. The agent:

Reads the task

Reads the email

Reads all available tools (names + descriptions)

Decides which ones are applicable

The LLM powers this decision-making, not any hardcoded rule.

In [6]:
email_task = Task(
    description = f''' Take the following rough email and rewrite it into a professional and polished
    version.
    Expand abbreviation:
    """{original_email}""" ''',
    agent = email_assistant,
    expected_output= " A professional writtent email with proper formatting and content."
)

crew = Crew(
    agents = [email_assistant],
    tasks=[email_task],
    verbose = True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 556c2f2b-eb21-440d-83cd-0c272f5e3271                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Task:  Take the following rough email and rewrite it into a professional and polished                          │
│      version.                                                                                                   │
│      Expand abbreviation:                                                                                       │
│      """ looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.              │
│  Let's sync up tomorrow if SYNBOT allows. ping me if any blockers.                                              │
│  """                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

d:\ML Lab\crewai-demo\.venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Thought: I need to rewrite the given email to make it sound more professional and clear. I should expand       │
│  abbreviations and use more formal language. I will use the Replace Jargons tool to achieve this.               │
│                                                                                                                 │
│  Using Tool: Replace Jargons                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday. Let'  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'                                   │
│  Consider replacing 'TAS' with 'technical architecture stack'                                                   │
│  Consider replacing 'SDS' with 'Smart Data Syncer'                                                              │
│  Consider replacing 'ping' with 'reach out'                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Including Priya in this email. Updates on the technical architecture stack and Project Phoenix (internal AI    │
│  revamp project) are in the deck. The estimated time of arrival for Smart Data Syncer integration is Friday.    │
│                                                                                                                 │
│  Let's coordinate tomorrow if the Scheduling System allows. Please reach out to me if there are any             │
│  impediments.                                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a90e2c8e-d70e-42a8-b5fd-aabcb6024508                                                                     │
│  Agent: Email Assistant Agent                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 556c2f2b-eb21-440d-83cd-0c272f5e3271                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Including Priya in this email. Updates on the technical architecture stack and Project Phoenix   │
│  (internal AI revamp project) are in the deck. The estimated time of arrival for Smart Data Syncer integration  │
│  is Friday.                                                                                                     │
│                                                                                                                 │
│  Let's coordinate tomorrow if the Scheduling System allows. Please reach out to me if there are any             │
│  impediments.                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Including Priya in this email. Updates on the technical architecture stack and Project Phoenix (internal AI revamp project) are in the deck. The estimated time of arrival for Smart Data Syncer integration is Friday.

Let's coordinate tomorrow if the Scheduling System allows. Please reach out to me if there are any impediments.
